1. load data
2. preprocessing
3. build models
4. test and improve

In [1]:
# libraries
library(tidyverse) # metapackage of all tidyverse packages
library(keras)
library(randomForest)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin




In [2]:
data <- read_csv("../input/bluebook-for-bulldozers/TrainAndValid.csv")
head(data)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  SalesID = col_double(),
  SalePrice = col_double(),
  MachineID = col_double(),
  ModelID = col_double(),
  datasource = col_double(),
  auctioneerID = col_double(),
  YearMade = col_double(),
  MachineHoursCurrentMeter = col_double()
)
ℹ Use `spec()` for the full column specifications.




SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,⋯,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1139246,66000,999089,3157,121,3,2004,68,Low,11/16/2006 0:00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Standard,Conventional
1139248,57000,117657,77,121,3,1996,4640,Low,3/26/2004 0:00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Standard,Conventional
1139249,10000,434808,7009,121,3,2001,2838,High,2/26/2004 0:00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1139251,38500,1026470,332,121,3,2001,3486,High,5/19/2011 0:00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1139253,11000,1057373,17311,121,3,2007,722,Medium,7/23/2009 0:00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1139255,26500,1001274,4605,121,3,2004,508,Low,12/18/2008 0:00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Preprocessing

In [3]:
# drop columns with IDs
colIDs <- c("auctioneerID", "SalesID", "ModelID", "MachineID")
data <- data %>% select(-all_of(colIDs)) 


In [4]:
# change saledate to saleyear
data$saleYear <- as.numeric(format(as.Date(data$saledate, format = "%m/%d/%Y"),"%Y"))
data <- subset(data, select = -c(saledate))

In [5]:
dim(data)

#check for NA values percentage
enframe(colSums(is.na(data)) / nrow(data))

# choose variables with less than 75% missing values
non.mis <- enframe(colSums(is.na(data))/nrow(data)) %>% filter(value < 0.75) %>% select(name) %>% pull
data <- data[, non.mis]
dim(data)
# only 21 variables remains in the dataset

[1] 412698     49

name,value
<chr>,<dbl>
SalePrice,0.0000000000
datasource,0.0000000000
YearMade,0.0000000000
MachineHoursCurrentMeter,0.6425861041
UsageBand,0.8214917446
fiModelDesc,0.0000000000
fiBaseModel,0.0000000000
fiSecondaryDesc,0.3409926871
fiModelSeries,0.8578452040


[1] 412698     21

In [6]:
# find columns with numerical values (there are 5)
## NAs
colSums(is.na(dplyr::select_if(data, is.numeric)))
# only 1 variable (MachineHoursCurrentMeter) contains NAs

SalePrice               datasource                 YearMade 
                       0                        0                        0 
MachineHoursCurrentMeter                 saleYear 
                  265194                        0

In [7]:
median(data$MachineHoursCurrentMeter, na.rm = T)
# median of the MachineHoursCurrentMeter is 0
(sum(is.na(data$MachineHoursCurrentMeter)) + sum(data$MachineHoursCurrentMeter == 0, na.rm = T)) / nrow(data)
# filling NAs with median would end up in 82% observations having the value of 0
# thus we won't use this variable
data <- data %>% select(-"MachineHoursCurrentMeter")

[1] 0

[1] 0.8214917

Distinguishing between text and categorical variables

In [8]:
# data %>% select_if(is.character) %>% ncol()  # num. of string variables = 16
# variables having > 20 categories
data %>% summarise(across(where(is.character), ~ length(unique(.x))))
not.categ <- data %>% summarise(across(where(is.character), ~ length(unique(.x)))) %>%
    select_if(function(x) x > 20) %>% names()

# choose only variables with < 20 categories and transform them into factor     
data <- data %>% select(-all_of(not.categ)) %>% mutate_if(is.character, as.factor)

fiModelDesc,fiBaseModel,fiSecondaryDesc,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Ride_Control,Transmission,Hydraulics,Ripper,Coupler
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
5059,1961,173,7,74,53,6,6,5,7,3,4,9,13,5,4


In [9]:
# see the categories with their frequency of categ. variables
categ <- data %>% select_if(is.factor) %>% names()
for(i in 1:length(categ)){data[, categ[i]] %>% summary() %>% print()}


         ProductSize    
 Compact       :  6280  
 Large         : 21396  
 Large / Medium: 51297  
 Medium        : 64342  
 Mini          : 25721  
 Small         : 27057  
 NA's          :216605  
 ProductGroup
 BL : 81401  
 MG : 26258  
 SSL: 45011  
 TEX:104230  
 TTT: 82582  
 WL : 73216  
            ProductGroupDesc 
 Backhoe Loaders    : 81401  
 Motor Graders      : 26258  
 Skid Steer Loaders : 45011  
 Track Excavators   :104230  
 Track Type Tractors: 82582  
 Wheel Loader       : 73216  
           Drive_System   
 All Wheel Drive :   824  
 Four Wheel Drive: 33551  
 No              : 25166  
 Two Wheel Drive : 47546  
 NA's            :305611  
               Enclosure     
 EROPS              :141769  
 EROPS AC           :    18  
 EROPS w AC         : 92601  
 NO ROPS            :     3  
 None or Unspecified:     2  
 OROPS              :177971  
 NA's               :   334  
                 Forks       
 None or Unspecified:183061  
 Yes                : 14654  


In [10]:
# categorical variables, number of NAs
enframe(colSums(is.na(dplyr::select_if(data, is.factor))))

# function to find the most frequent value of factor
calc_mode <- function(x) {
  uniqx <- unique(na.omit(x))
  return(uniqx[which.max(tabulate(match(x, uniqx)))])
}

# fill NAs with the most frequent value
data <- data %>% mutate_if(is.factor, ~ifelse(is.na(.), calc_mode(.), .))

name,value
<chr>,<dbl>
ProductSize,216605
ProductGroup,0
ProductGroupDesc,0
Drive_System,305611
Enclosure,334
Forks,214983
Ride_Control,259970
Transmission,224691
Hydraulics,82565


In [11]:
# Final check for NAs
enframe(colSums(is.na(data)) / nrow(data))

# Final check for data dimension
dim(data)

# Final variables
names(data)

name,value
<chr>,<dbl>
SalePrice,0
datasource,0
YearMade,0
ProductSize,0
ProductGroup,0
ProductGroupDesc,0
Drive_System,0
Enclosure,0
Forks,0


[1] 412698     15

[1] "SalePrice"        "datasource"       "YearMade"         "ProductSize"     
 [5] "ProductGroup"     "ProductGroupDesc" "Drive_System"     "Enclosure"       
 [9] "Forks"            "Ride_Control"     "Transmission"     "Hydraulics"      
[13] "Ripper"           "Coupler"          "saleYear"

In [12]:
# prepare train and valid data
div <- rbinom(nrow(data), 1, 0.25)
train <- data[div == 0, ]
valid <- data[div == 1, ]
dim(train)
dim(valid)

[1] 309903     15

[1] 102795     15

# Random forest model

In [13]:
#rf_classifier = randomForest(SalePrice ~ ., data = train, na.action = na.omit)
model <- randomForest(SalePrice ~ ., data = train, na.action = na.omit, ntree = 50, mtry = 4)

In [14]:
model
# Var explained: 70.5%, mean of sq. residuals: 1581322253
saveRDS(model, "./RFmodel.rds")


Call:
 randomForest(formula = SalePrice ~ ., data = train, ntree = 50,      mtry = 4, na.action = na.omit) 
               Type of random forest: regression
                     Number of trees: 50
No. of variables tried at each split: 4

          Mean of squared residuals: 157798350
                    % Var explained: 70.51

In [15]:
# prediction
pred <- predict(model, valid, type = "class") %>% cbind(valid$SalePrice, .)
head(pred)
write.table(pred, "./predictions.csv", sep = ";")


,,.
1,10000,12545.33
2,13500,13936.14
3,9500,13905.63
4,34500,32586.63
5,33000,28959.32
6,15500,17934.73
